In [1]:
midi_lib_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/midi_lib"

output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output"


In [2]:
topk15_t1_0_ngram0_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0"

In [3]:
model_result_str = "s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-78 d-6 v-20 o-12 t-35 i-0 p-55 d-17 v-20"

In [4]:
import sys

sys.path.append(midi_lib_path)

In [5]:
import os

from converter.midi_dto_to_midi_file_object import midi_dto_to_midi_file_object_converter
from converter.musecoco_line_to_midi_dto import musecoco_line_to_midi_dto_converter

from midi_file_utils import write_midi_file

In [6]:
abbreviations = {
    "b": "bar",
    "o": "position",
    "s": "time_signature",
    "t": "tempo",
    "i": "instrument",
    "p": "pitch",
    "d": "duration",
    "v": "velocity",
    "n": "pitch_name",
    "c": "pitch_octave",
    "f": "family",
    "e": "special"
}

In [7]:
def midi_number_to_note_name(midi_number):
    '''
        Convert a midi number to a note name
    '''
    notes = [
        "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"
    ]

    note = notes[midi_number % 12]
    octave = midi_number // 12 - 1

    return f"{note}{octave}"

In [8]:
def print_key_value_line(key_value_line):
    if key_value_line == None or key_value_line == []:
        print("None or Empty")
        print(key_value_line)
        return
    
    n_space_for_digits = 5

    n_letter_per_print_time_for_abbrs = {
        x : len(abbreviations[x]) + n_space_for_digits
        for x in abbreviations
    }

    for pair in key_value_line:
        key, value = pair

        converted_value = None

        if abbreviations[key] == 'pitch':
            converted_value = int(value)
            converted_value = midi_number_to_note_name(int(value))
        else:
            converted_value = ''

        abbr_meaning = abbreviations[key]

        pair_str = f"{abbr_meaning}: {value} {converted_value}"
        end_spaces = ' ' * (n_letter_per_print_time_for_abbrs[key] - len(abbr_meaning) - len(str(value)))

        if abbr_meaning in ['position', 'time_signature']:
            print()

        print(pair_str, end=end_spaces)

In [9]:
# CODE SNIPPET: Convert string to token list

model_result_tokens_list = model_result_str.split(" ")

model_result_tokens_list = list(filter(lambda x: "-" in x, model_result_tokens_list))

model_result_tokens_list = list(map(lambda x: tuple([x.split("-")[0], int(x.split("-")[1])]), model_result_tokens_list))

print(model_result_tokens_list)

[('s', 8), ('o', 0), ('t', 35), ('i', 0), ('p', 67), ('d', 6), ('v', 20), ('o', 6), ('t', 35), ('i', 0), ('p', 78), ('d', 6), ('v', 20), ('o', 12), ('t', 35), ('i', 0), ('p', 55), ('d', 17), ('v', 20)]


In [10]:
print_key_value_line(model_result_tokens_list)


time_signature: 8     
position: 0     tempo: 35    instrument: 0     pitch: 67 G4   duration: 6     velocity: 20    
position: 6     tempo: 35    instrument: 0     pitch: 78 F#5   duration: 6     velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 55 G3   duration: 17    velocity: 20    

In [11]:
# CODE SNIPPET: Convert token list to MIDI

output_midi_file_name = "musecoco_result.mid"

output_midi_file_path = f"{output_dir}/{output_midi_file_name}"

midi_dto = musecoco_line_to_midi_dto_converter(
    model_result_tokens_list,
    tick_per_beat=480
)

midi_obj = midi_dto_to_midi_file_object_converter(midi_dto)

write_midi_file(midi_obj, output_midi_file_path)

MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output/musecoco_result.mid


In [12]:
topk15_t1_0_ngram0_output_dir = f"{output_dir}/topk15-t1.0-ngram0"

# Create the output directory if it does not exist
if not os.path.exists(topk15_t1_0_ngram0_output_dir):
    os.makedirs(topk15_t1_0_ngram0_output_dir)
else:
    pass

In [13]:
# Listing all 0.txt and 1.txt files in the topk15_t1_0_ngram0 directory recursively
topk15_t1_0_ngram0_txt_files = []

for root, dirs, files in os.walk(topk15_t1_0_ngram0_dir):
    for file in files:
        if file.endswith("0.txt") or file.endswith("1.txt"):
            topk15_t1_0_ngram0_txt_files.append(os.path.join(root, file))

# Sort by alphabetical order
topk15_t1_0_ngram0_txt_files = sorted(topk15_t1_0_ngram0_txt_files)

print(len(topk15_t1_0_ngram0_txt_files))

135


In [14]:
n_success = 0

for file_path in topk15_t1_0_ngram0_txt_files:
    print(f"PROCESSING: {file_path}")
    
    with open(file_path, "r") as file:
        lines = file.readlines()

        for line in lines:
            line = line.split("<sep>")[1].strip()
            
            tokens = line.split(" ")
            tokens = list(filter(lambda x: "-" in x, tokens))
            tokens = list(map(lambda x: tuple([x.split("-")[0], int(x.split("-")[1])]), tokens))

            try:
                midi_dto = musecoco_line_to_midi_dto_converter(
                    tokens,
                    tick_per_beat=480
                )

                midi_obj = midi_dto_to_midi_file_object_converter(midi_dto)
                
                file_name = file_path.split('/')[-1].split('.')[0]
                grand_parent_dir_name = file_path.split('/')[-3]
                output_file_path = f"{topk15_t1_0_ngram0_output_dir}/{grand_parent_dir_name}_{file_name}.mid"
                
                write_midi_file(midi_obj, output_file_path)
                
                n_success += 1
            except Exception as e:
                print_key_value_line(tokens)                

print()
print(f"SUCCESS: {n_success}/{len(topk15_t1_0_ngram0_txt_files)}")                

PROCESSING: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0/1/remi/0.txt
item: ('v', 20)

time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 37 C#2   duration: 17    velocity: 20    pitch: 52 E3   duration: 6     velocity: 20    
position: 42    tempo: 35    instrument: 12    velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 55 G3   duration: 6     velocity: 20    
position: 6     tempo: 35    instrument: 0     pitch: 71 B4   duration: 6     velocity: 20    
position: 24    tempo: 35    
position: 18    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6     velocity: 20    pitch: 62 D4   duration: 6     velocity: 20    pitch: 55 G3   duration: 6     velocity: 20    pitch: 64 E4   duration: 6     velocity: 20    pitch: 52 E3   duration: 18    velocity: 20    
position: 24    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6     velocity: 2